In [1]:
#imports
import folium
from folium.plugins import MarkerCluster
from dotenv import load_dotenv
import os
data_folder = os.getenv("dataFolder")
staticFolder = os.getenv("staticFolder")

In [2]:
def read_files(
    file_locations: str, file_blobs: str
) -> tuple[list[tuple[float, float, int]], list[tuple[float, float, float]]]:
    loc_list = []
    blob_list = []
    with open(file_locations, "r") as file:
        for line in file:
            # Remove any surrounding whitespace and split by comma
            parts = line.strip().split(",")
            # Convert parts to the appropriate types and create a tuple
            tuple_item = (float(parts[0]), float(parts[1]), int(parts[2]))
            loc_list.append(tuple_item)

    with open(file_blobs, "r") as file:
        for line in file:
            # Remove any surrounding whitespace and split by comma
            parts = line.strip().split(",")
            # Convert parts to the appropriate types and create a tuple
            tuple_item = (float(parts[0]), float(parts[1]), float(parts[2]))
            blob_list.append(tuple_item)

    return loc_list, blob_list

In [3]:
good_trace = []
with open(f"{data_folder}/good_trace", "r") as file:
    for line in file:
        # Remove any surrounding whitespace and split by comma
        parts = line.strip().split(",")
        # Convert parts to the appropriate types and create a tuple
        tuple_item = (float(parts[0]), float(parts[1]), int(parts[2]))
        good_trace.append(tuple_item)
good_trace = [(lat, lon) for lat, lon, _ in good_trace]

In [4]:
radiuses = [0, 50, 100, 200, 400, 800, 1600]
time_deltas = [0, 60, 300, 600, 1800, 3600]

for radius in radiuses:
    for time_delta in time_deltas:
        loc_file = f"{data_folder}/obfuscated/{radius}_{time_delta}_res"
        blob_file = f"{data_folder}/obfuscated/{radius}_{time_delta}_blobs"
        locations, blobs = read_files(loc_file, blob_file)
        average_lat = sum([t[0] for t in locations]) / len(locations)
        average_lon = sum([t[1] for t in locations]) / len(locations)
        map = folium.Map(
            location=[average_lat, average_lon],
            zoom_start=13,
        )

        folium.PolyLine(
            locations=good_trace,
            color="#FF0000",
            tooltip="Real location",
        ).add_to(map)

        folium.PolyLine(
            locations=[(lat, lon) for lat, lon, _ in locations],
            color="#0000FF",
            tooltip="Obfuscated location",
        ).add_to(map)

        for blob in blobs:
            folium.Circle(
                location=[blob[0], blob[1]],
                radius=blob[2],
                fill_opacity=0.2,
                fill_color="cornflowerblue",
            ).add_to(map)
        map.save(f"{staticFolder}/obfuscated/{radius}_{time_delta}.html")